# 字典的扩展

除了默认的dict以外，在标准库里 `collections` 模块中，还提供了一些默认dict的扩展。

## defaultdict

在用户创建defaultdict的对象的时候，就需要给它配置一个为找不到的键创造默认值的方法。

具体而言，在实例化一个defaultdict的时候，需要给构造方法提供一个可调用对象，这个可调用对象会在 `__getitem__` 碰到找不到的键的时候被调用，让 `__getitem__` 返回某种默认值。

如，我们新建一个这样的字典：`dd = defaultdict(list)` ，如果使用 `dd[k]` 去获取 k 对应的值时， k 不存在，则会有以下步骤：

1. 调用 `list()` 来建立一个新列表
2. 把这个新列表作为值， k 作为它的键，放在 dd 中。
3. 返回这个列表的引用

而这个用来生成默认值的可调用对象存放在名为 `default_factory` 的实例属性里

实际上这一切的背后是由于特殊方法 `__missing__` ，它会在找不到键的情况下被调用，然后在它里面调用 `default_factory`

所以如果我们需要自定义映射类型时，需要处理或兼容一些找不到或特殊情况，可以考虑实现该方法。

## OrderedDict

这个类型在添加键的时候会保持顺序，因此键的迭代次序总是一致的。

OrderedDict的 `pipitem` 方法默认返回并删除字典的最后一个元素，但是也可以通过指定 `lst=False` 来指定从前面开始出栈。

In [1]:
from collections import OrderedDict

od = OrderedDict({'one': 1, 'two': 2, 'three': 3})
od

OrderedDict([('one', 1), ('two', 2), ('three', 3)])

In [2]:
od.popitem()

('three', 3)

In [3]:
od.popitem(last=False)

('one', 1)

## ChainMap

该类型可以容纳多个不同的映射对象，然后在进行键查找操作的时候，这些对象会被当作一个整体被逐一查找，直到键被找到为止。

In [10]:
from collections import ChainMap

cm = ChainMap(locals(), globals())
cm.get('__name__')

'__main__'

## Counter

这个映射类型会给键准备一个整数计数器。每次更新一个键的时候都会增加这个计数器。

In [11]:
from collections import Counter

ct = Counter('abraccadbs')
ct

Counter({'a': 3, 'b': 2, 'r': 1, 'c': 2, 'd': 1, 's': 1})

In [12]:
ct.update('aaaaa')
ct

Counter({'a': 8, 'b': 2, 'r': 1, 'c': 2, 'd': 1, 's': 1})

## UserDict

这个类其实是把标准 dict 用纯Python重新实现了一遍，一般来说不建议直接继承 dict ，而是建议使用 UserDict 来实现我们自己的映射类型。

UserDict 有一个叫做data的属性，是dict的实例，这个属性是UserDict最终存储数据的地方。

如下面的例子中，将字典的键通通存为 str ，但是同时需要支持通过数字和字符的查询操作：

In [13]:
from collections import UserDict

class StrKeyDict(UserDict):
    
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key):
        return str(key) in self.data
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item

In [14]:
sd = StrKeyDict([('2', 'two'), ('4', 'four')])
sd

{'2': 'two', '4': 'four'}

In [16]:
sd['2']

'two'

In [17]:
sd[4]

'four'

In [21]:
sd.update({1: 'one', 3: 'three'})
sd

{'2': 'two', '4': 'four', '1': 'one', '3': 'three'}

In [22]:
sd.get(1)

'one'